In [18]:
import pandas as pd
import numpy as np
import requests,  os, string
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install BeautifulSoup4
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from urllib.request import urlopen
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

Now we create a dictionary to hold 3 keys PostalCode, Borough and Neighborhood, such that:

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A  is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
- If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [23]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Check: neihbourhoods with M5A postcode have been joined

In [24]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Check: rows with duplicates postcodes >> after removing duplicate postcodes the row count is the same as before (no duplicates hence no need to group by postcode)

In [25]:
duplicates=df.duplicated(['PostalCode']) # returns boolean series
duplicates[duplicates[:]] # removing duplicate postcodes from boolean series
duplicates.shape #check size is the same as df.shape (see above)

(103,)

Number of rows of dataframe

In [26]:
df.shape

(103, 3)

In [27]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Part 2
Now that you have built a dataframe of the postal code of each neighbourhood along with the borough name and neighbourhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighbourhood. 

In [41]:
#dfgeo = pd.read_csv("https://cocl.us/Geospatial_data") #using link to access the csv file
dfgeo = pd.read_csv(r'Geospatial_Coordinates.csv')
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [42]:
dfgeo

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [51]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [52]:
df2.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Part 3
Explore and cluster the neighbourhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

In [53]:
#Foursquare credentials
CLIENT_ID = '2P4NUK2Z1FNXQKV4RBJM1RGQVRV4AGBWDHCMTRI54UPCDFPO'# Foursquare Id
CLIENT_SECRET = 'KHFPIIS0VT1LPW3MQMWYL2DEVMY1CF2GLAQZRAQYRIB4FNB2' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [54]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M5A']['Latitude']
neighborhood_longitude = df2.loc['M5A']['Longitude']

In [55]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2P4NUK2Z1FNXQKV4RBJM1RGQVRV4AGBWDHCMTRI54UPCDFPO&client_secret=KHFPIIS0VT1LPW3MQMWYL2DEVMY1CF2GLAQZRAQYRIB4FNB2&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [56]:
results = requests.get(url).json()

In [57]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we will clean the json and structure it into a pandas dataframe

In [58]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


Exploring Boroughs in Toronto
- Repeat for all neighborhoods in Toronto

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude']) # working with boroughs

North York
North York
Downtown Toronto
North York
Queen's Park
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York/East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
W

In [63]:
print(venues.shape)

venues.head()

(2117, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [64]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,103,103,103,103,103,103
Downtown Toronto,1095,1095,1095,1095,1095,1095
Downtown Toronto Stn A,99,99,99,99,99,99
East Toronto,103,103,103,103,103,103
East Toronto Business,17,17,17,17,17,17
East York,73,73,73,73,73,73
East York/East Toronto,4,4,4,4,4,4
Etobicoke,71,71,71,71,71,71
Etobicoke Northwest,4,4,4,4,4,4


In [65]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 276 unique categories.


Analysing each neighborhood

In [69]:
#one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size

In [70]:
onehot.shape

(2117, 276)

Group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [71]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

Let's confirm the new size

In [72]:
grouped.shape

(15, 276)

The Top 5 venues per neighborhood

In [73]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0  Coffee Shop     0.08
1  Sandwich Place  0.07
2  Café            0.06
3  Park            0.06
4  Pizza Place     0.05


----Downtown Toronto----
                venue  freq
0  Coffee Shop         0.10
1  Café                0.05
2  Hotel               0.03
3  Restaurant          0.03
4  Italian Restaurant  0.02


----Downtown Toronto Stn A----
                venue  freq
0  Coffee Shop         0.11
1  Seafood Restaurant  0.04
2  Restaurant          0.03
3  Bakery              0.03
4  Cocktail Bar        0.03


----East Toronto----
                venue  freq
0  Greek Restaurant    0.08
1  Coffee Shop         0.07
2  Italian Restaurant  0.05
3  Ice Cream Shop      0.04
4  Brewery             0.04


----East Toronto Business----
                venue  freq
0  Light Rail Station  0.12
1  Yoga Studio         0.06
2  Auto Workshop       0.06
3  Comic Shop          0.06
4  Park                0.06


----East York----
                 venue  f

Let's put that into a pandas dataframe

In [74]:
# write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues): 
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Display the Top 10 venues per neighborhood

In [75]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Dessert Shop,Clothing Store,Restaurant,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Sushi Restaurant,Gym
2,Downtown Toronto Stn A,Coffee Shop,Seafood Restaurant,Hotel,Restaurant,Café,Beer Bar,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Bakery
3,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Bookstore,Restaurant,Pub,Café,American Restaurant
4,East Toronto Business,Light Rail Station,Yoga Studio,Garden,Comic Shop,Pizza Place,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
5,East York,Coffee Shop,Bank,Sandwich Place,Sporting Goods Shop,Pizza Place,Burger Joint,Intersection,Beer Store,Pet Store,Park
6,East York/East Toronto,Pizza Place,Park,Convenience Store,Intersection,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Event Space,Dim Sum Restaurant
7,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Park,Fast Food Restaurant,Liquor Store,Gym,Bakery
8,Etobicoke Northwest,Bar,Rental Car Location,Drugstore,Truck Stop,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
9,Mississauga,Coffee Shop,Hotel,Gym,Burrito Place,Sandwich Place,Middle Eastern Restaurant,Gas Station,Intersection,Fried Chicken Joint,American Restaurant


In [76]:
# set number of clusters
kclusters = 5 

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 4, 0, 2, 3], dtype=int32)

Generate k-means clusters

In [77]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# add latitude/longitude for each neighborhood
df3 = venues.groupby('Neighborhood').first().reset_index() #group by and take first row
merged = df3[['Neighborhood', 'Neighborhood Longitude', 'Neighborhood Latitude']].join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged = merged.rename({'Neighborhood Longitude': 'Longitude', 'Neighborhood Latitude': 'Latitude'}, axis=1)  # rename coloumns long/lat
merged

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,-79.388790,43.728020,0,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Dessert Shop,Clothing Store,Restaurant,Gym
1,Downtown Toronto,-79.360636,43.654260,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Sushi Restaurant,Gym
2,Downtown Toronto Stn A,-79.374846,43.646435,0,Coffee Shop,Seafood Restaurant,Hotel,Restaurant,Café,Beer Bar,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Bakery
3,East Toronto,-79.293031,43.676357,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Bookstore,Restaurant,Pub,Café,American Restaurant
4,East Toronto Business,-79.321558,43.662744,0,Light Rail Station,Yoga Studio,Garden,Comic Shop,Pizza Place,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
5,East York,-79.309937,43.706397,0,Coffee Shop,Bank,Sandwich Place,Sporting Goods Shop,Pizza Place,Burger Joint,Intersection,Beer Store,Pet Store,Park
6,East York/East Toronto,-79.338106,43.685347,4,Pizza Place,Park,Convenience Store,Intersection,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Event Space,Dim Sum Restaurant
7,Etobicoke,-79.554724,43.650943,0,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Park,Fast Food Restaurant,Liquor Store,Gym,Bakery
8,Etobicoke Northwest,-79.594054,43.706748,2,Bar,Rental Car Location,Drugstore,Truck Stop,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
9,Mississauga,-79.615819,43.636966,3,Coffee Shop,Hotel,Gym,Burrito Place,Sandwich Place,Middle Eastern Restaurant,Gas Station,Intersection,Fried Chicken Joint,American Restaurant


In [78]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ON_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [80]:
print(merged.shape)
merged.sort_values(["Cluster Labels"], inplace=True)
merged.head(20)

(15, 14)


,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,-79.388790,43.728020,0,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Dessert Shop,Clothing Store,Restaurant,Gym
1,Downtown Toronto,-79.360636,43.654260,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Sushi Restaurant,Gym
2,Downtown Toronto Stn A,-79.374846,43.646435,0,Coffee Shop,Seafood Restaurant,Hotel,Restaurant,Café,Beer Bar,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Bakery
3,East Toronto,-79.293031,43.676357,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Bookstore,Restaurant,Pub,Café,American Restaurant
4,East Toronto Business,-79.321558,43.662744,0,Light Rail Station,Yoga Studio,Garden,Comic Shop,Pizza Place,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
5,East York,-79.309937,43.706397,0,Coffee Shop,Bank,Sandwich Place,Sporting Goods Shop,Pizza Place,Burger Joint,Intersection,Beer Store,Pet Store,Park
7,Etobicoke,-79.554724,43.650943,0,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Park,Fast Food Restaurant,Liquor Store,Gym,Bakery
10,North York,-79.329656,43.753259,0,Coffee Shop,Clothing Store,Restaurant,Bank,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Pizza Place,Sandwich Place,Park
11,Queen's Park,-79.389494,43.662301,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,College Cafeteria
12,Scarborough,-79.194353,43.806686,0,Chinese Restaurant,Fast Food Restaurant,Bank,Bakery,Coffee Shop,Indian Restaurant,Pharmacy,Breakfast Spot,Intersection,Pizza Place


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

Cluster 1

In [81]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Dessert Shop,Clothing Store,Restaurant,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Sushi Restaurant,Gym
2,Downtown Toronto Stn A,Coffee Shop,Seafood Restaurant,Hotel,Restaurant,Café,Beer Bar,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Bakery
3,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Bookstore,Restaurant,Pub,Café,American Restaurant
4,East Toronto Business,Light Rail Station,Yoga Studio,Garden,Comic Shop,Pizza Place,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
5,East York,Coffee Shop,Bank,Sandwich Place,Sporting Goods Shop,Pizza Place,Burger Joint,Intersection,Beer Store,Pet Store,Park
7,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Park,Fast Food Restaurant,Liquor Store,Gym,Bakery
10,North York,Coffee Shop,Clothing Store,Restaurant,Bank,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Pizza Place,Sandwich Place,Park
11,Queen's Park,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,College Cafeteria
12,Scarborough,Chinese Restaurant,Fast Food Restaurant,Bank,Bakery,Coffee Shop,Indian Restaurant,Pharmacy,Breakfast Spot,Intersection,Pizza Place


Cluster 2

In [82]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,York,Park,Caribbean Restaurant,Trail,Skating Rink,Field,Jewelry Store,Sandwich Place,Discount Store,Tennis Court,Pool


Cluster 3

In [83]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Etobicoke Northwest,Bar,Rental Car Location,Drugstore,Truck Stop,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run


Cluster 4

In [84]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Mississauga,Coffee Shop,Hotel,Gym,Burrito Place,Sandwich Place,Middle Eastern Restaurant,Gas Station,Intersection,Fried Chicken Joint,American Restaurant


Cluster 5

In [85]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[0] + list(range(4, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,East York/East Toronto,Pizza Place,Park,Convenience Store,Intersection,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Event Space,Dim Sum Restaurant


End of code